# Gratificación instantánea

## ¡Tu primer proyecto de LLM de Frontier!

Construyamos una solución LLM útil en cuestión de minutos.

Al finalizar este curso, habrás creado una solución de IA autónoma de Agentic con 7 agentes que colaboran para resolver un problema comercial. ¡Todo a su debido tiempo! Comenzaremos con algo más pequeño...

Nuestro objetivo es codificar un nuevo tipo de navegador web. Dale una URL y responderá con un resumen. ¡El Reader's Digest de Internet!

Antes de comenzar, deberías haber completado la configuración para [PC](../SETUP-PC.md) o [Mac](../SETUP-mac.md) y, con suerte, habrás iniciado este laboratorio de Jupyter desde el directorio raíz del proyecto, con tu entorno activado.

## Si eres nuevo en Jupyter Lab

¡Bienvenido al maravilloso mundo de la experimentación en ciencia de datos! Una vez que hayas usado Jupyter Lab, te preguntarás cómo pudiste vivir sin él. Simplemente haga clic en cada "celda" que contenga código, como la celda inmediatamente debajo de este texto, y presione Shift+Return para ejecutar esa celda. Como desee, puede agregar una celda con el botón + en la barra de herramientas e imprimir valores de variables o probar variaciones.

He escrito un cuaderno llamado [Guía para Jupyter](Guide%20to%20Jupyter.ipynb) para ayudarlo a familiarizarse más con Jupyter Labs, incluida la adición de comentarios de Markdown, el uso de `!` para ejecutar comandos de shell y `tqdm` para mostrar el progreso.

Si prefiere trabajar en IDE como VSCode o Pycharm, ambos funcionan muy bien con estos cuadernos de laboratorio también.

## Si desea repasar su Python

He agregado un cuaderno llamado [Python intermedio](Intermediate%20Python.ipynb) para ayudarlo a ponerse al día. Pero no deberías probarlo si ya tienes una buena idea de lo que hace este código:
`yield from {book.get("author") for book in books if book.get("author")}`

## Estoy aquí para ayudarte

Si tienes algún problema, no dudes en ponerte en contacto conmigo.
Estoy disponible a través de la plataforma,o en https://www.linkedin.com/in/juan-gabriel-gomila-salas/ si quieres conectarte (¡y me encanta conectarme!).

## Más resolución de problemas

Consulta el cuaderno de [solución de problemas](troubleshooting.ipynb) en esta carpeta para diagnosticar y solucionar problemas comunes.

## ¡Si esto ya es pan comido!

Si ya te sientes cómodo con el material de hoy, no te desanimes; puedes avanzar rápidamente en los primeros laboratorios; profundizaremos mucho más a medida que avancen las semanas.

## Valor comercial de estos ejercicios

Una última reflexión. Aunque he diseñado estos cuadernos para que sean educativos, también he intentado que sean divertidos. Haremos cosas divertidas, como que los estudiantes de LLM cuenten chistes y discutan entre ellos. Pero, fundamentalmente, mi objetivo es enseñar habilidades que se puedan aplicar en los negocios. Explicaré las implicaciones comerciales a medida que avancemos y vale la pena tener esto en cuenta: a medida que adquiera experiencia con modelos y técnicas, piense en formas en las que podría poner esto en práctica en el trabajo hoy. Comuníquese conmigo si desea hablar más sobre esto o si tiene ideas para compartir conmigo.

In [ ]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# Si recibe un error al ejecutar esta celda, diríjase al cuaderno de solución de problemas (troubleshooting.ipynb).

# Conexión a OpenAI

La siguiente celda es donde cargamos las variables de entorno en su archivo `.env` y nos conectamos a OpenAI.

## Solución de problemas si tiene problemas:

Vaya al cuaderno [solución de problemas](troubleshooting.ipynb) en esta carpeta para obtener el código paso a paso para identificar la causa raíz y solucionarlo.

Si realiza un cambio, intente reiniciar el "Kernel" (el proceso de Python que se encuentra detrás de este cuaderno) mediante el menú Kernel >> Reiniciar Kernel y borrar los resultados de todas las celdas. Luego, intente con este cuaderno nuevamente, comenzando desde arriba.

O bien, ¡contácteme! Envíeme un mensaje a través del foro del curso y haremos que esto funcione.

¿Tiene alguna inquietud sobre los costos de la API? Vea mis notas en el README: los costos deberían ser mínimos y puede controlarlos en cada punto. También puede usar Ollama como una alternativa gratuita, que analizamos durante el día 2.

In [ ]:
# Cargamos las variables del fichero .env

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No se encontró ninguna clave API: diríjase al cuaderno de resolución de problemas en esta carpeta para identificarla y solucionarla.")
elif not api_key.startswith("sk-proj-"):
    print("Se encontró una clave API, pero no inicia sk-proj-; verifique que esté usando la clave correcta; consulte el cuaderno de resolución de problemas")
elif api_key.strip() != api_key:
    print("Se encontró una clave API, pero parece que puede tener espacios o caracteres de tabulación al principio o al final; elimínelos; consulte el cuaderno de resolución de problemas")
else:
    print("¡Se encontró la clave API y hasta ahora parece buena!")


In [ ]:
openai = OpenAI()

# Si esto no funciona, prueba con el menú Kernel >> Reiniciar Kernel y borrar las salidas de todas las celdas, luego ejecuta las celdas desde la parte superior de este cuaderno hacia abajo.
# Si TODAVÍA no funciona (¡qué horror!), consulta el cuaderno de resolución de problemas o prueba la siguiente línea:
# openai = OpenAI(api_key="your-key-here-starting-sk-proj-")

In [ ]:
# Una clase para representar una página web
# Si no estás familiarizado con las clases, consulta el cuaderno "Python intermedio"

class Website:
    """
    Una clase de utilidad para representar un sitio web que hemos scrappeado
    """

    def __init__(self, url):
        """
        Crea este objeto de sitio web a partir de la URL indicada utilizando la biblioteca BeautifulSoup
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No tiene título"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
# Vamos a probar

frog = Website("https://cursos.frogamesformacion.com")
print(frog.title)
print(frog.text)

## Tipos de indicaciones/prompts

Quizás ya sepas esto, pero si no, te resultará muy familiar.

Los modelos como GPT4o han sido entrenados para recibir instrucciones de una manera particular.

Esperan recibir:

**Una indicación del sistema** que les indique qué tarea están realizando y qué tono deben usar

**Una indicación del usuario**: el inicio de la conversación al que deben responder

In [ ]:
# Define nuestro mensaje de sistema: puedes experimentar con esto más tarde, cambiando la última oración a "Responder en Markdown en español".

system_prompt = "Eres un asistente que analiza el contenido de un sitio web \
y proporciona un breve resumen, ignorando el texto que podría estar relacionado con la navegación. \
Responder en Markdown."

In [ ]:
# Una función que escribe un mensaje de usuario que solicita resúmenes de sitios web:

def user_prompt_for(website):
    user_prompt = f"Estás viendo un sitio web titulado {website.title}"
    user_prompt += "\nEl contenido de este sitio web es el siguiente; \
    proporciona un breve resumen de este sitio web en formato Markdown. \
    Si incluye noticias, productos o anuncios, resúmelos también.\n\n"
    user_prompt += website.text
    return user_prompt

In [ ]:
print(user_prompt_for(frog))

## Mensajes

La API de OpenAI espera recibir mensajes en una estructura particular.
Muchas de las otras API comparten esta estructura:

```
[
    {"role": "system", "content": "el mensaje de sistema va aquí"},
    {"role": "user", "content": "el mensaje de usuario va aquí"}
]

In [ ]:
# Puedes ver cómo esta función crea exactamente el formato anterior

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [ ]:
messages_for(frog)

## Es hora de unirlo todo: ¡la API para OpenAI es muy simple!

In [ ]:
# Y ahora: llama a la API de OpenAI. ¡Te resultará muy familiar!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [ ]:
summarize("https://cursos.frogamesformacion.com")

In [ ]:
# Una función para mostrar esto de forma clara en la salida de Jupyter, usando markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [ ]:
display_summary("https://cursos.frogamesformacion.com")

# Probemos con más sitios web

Ten en cuenta que esto solo funcionará en sitios web que se puedan extraer con este enfoque simplista.

Los sitios web que se renderizan con Javascript, como las aplicaciones React, no se mostrarán. Consulta la carpeta de contribuciones de la comunidad para obtener una implementación de Selenium que solucione este problema. Deberás leer sobre la instalación de Selenium (¡pregunta a ChatGPT!).

Además, los sitios web protegidos con CloudFront (y similares) pueden generar errores 403. Muchas gracias, Andy J, por señalar esto.

¡Pero muchos sitios web funcionarán perfectamente!

In [ ]:
display_summary("https://cnn.com")

In [ ]:
display_summary("https://anthropic.com")

## Aplicaciones empresariales

En este ejercicio, experimentó la llamada a la API de un modelo de frontera (un modelo líder en la frontera de la IA) por primera vez. Esto se aplica ampliamente en todos los casos de uso de Gen AI y usaremos API como OpenAI en muchas etapas del curso, además de crear nuestros propios LLM.

Más específicamente, hemos aplicado esto a Resumen: un caso de uso clásico de Gen AI para hacer un resumen. Esto se puede aplicar a cualquier vertical empresarial: resumir las noticias, resumir el desempeño financiero, resumir un currículum en una carta de presentación; las aplicaciones son ilimitadas. Considere cómo podría aplicar Resumen en su negocio e intente crear un prototipo de solución.

## Un ejercicio extra para quienes disfrutan del web scraping

Puedes notar que si intentas `display_summary("https://openai.com")` - ¡no funciona! Esto se debe a que OpenAI tiene un sitio web sofisticado que usa Javascript. Hay muchas formas de evitar esto con las que algunos de ustedes pueden estar familiarizados. Por ejemplo, Selenium es un marco muy popular que ejecuta un navegador detrás de escena, renderiza la página y te permite consultarla. Si tienes experiencia con Selenium, Playwright o similar, no dudes en mejorar la clase Website para usarlos. En la carpeta de contribuciones de la comunidad, encontrarás una solución de Selenium de ejemplo de un estudiante (¡gracias!)

# Compartir tu código

¡Me encantaría que compartieras tu código después para que yo pueda compartirlo con otros! Notarás que algunos estudiantes ya han realizado cambios (incluida una implementación de Selenium) que encontrarás en la carpeta community-contributions. Si deseas agregar tus cambios a esa carpeta, envía una solicitud de incorporación de cambios con tus nuevas versiones en esa carpeta y yo fusionaré tus cambios.

Si no eres un experto en Git (¡y yo no lo soy!), entonces GPT ha dado algunas instrucciones útiles sobre cómo enviar una solicitud de incorporación de cambios. Es un proceso un poco complicado, pero una vez que lo hayas hecho una vez, estará bastante claro. Como consejo profesional: es mejor si borras las salidas de tus cuadernos Jupyter (Editar >> Limpiar las salidas de todas las celdas y luego Guardar) para tener cuadernos limpios.

Instrucciones de relaciones públicas cortesía de un amigo de IA: https://chatgpt.com/share/670145d5-e8a8-8012-8f93-39ee4e248b4c